## Task 2: Use an Encoder-Decoder recurrent neural network to add 2 numers


Inspiration from here: 
https://machinelearningmastery.com/learn-add-numbers-seq2seq-recurrent-neural-networks/

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
from random import seed
from random import randint
from numpy import array
from math import ceil
from math import log10
from math import sqrt
from numpy import argmax
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from sklearn.metrics import mean_squared_error
 

Using TensorFlow backend.


### Encoding

### Step 1: Transform digits to sequence of characters


alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' ']

#### Next: One hot encoding

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def encode_numbers(a, b, is_train=False, max_lenght=3):
  
  X = np.zeros(((max_lenght+1)*2, 11))
  Y = np.zeros((max_lenght+1, 11))

  if is_train:
    s = a+b
    sum_int = a+b
   
  a = [int(i) for i in str(a)]
  b = [int(i) for i in str(b)]
  
  a,b = pad_sequences([a,b],maxlen = max_lenght + 1, value=10)
#   print(a,b)
  input_nr = np.concatenate([a, b])
  
  
  for j, char in enumerate(input_nr):
      X[j, char] = 1
      
  if is_train:
    s = [int(i) for i in str(s)]
    a,b,s = pad_sequences([a,b,s],maxlen = max_lenght + 1,value=10)    
#     print(s)
    
    for j, char in enumerate(s):
       Y[j, char] = 1
    return X, Y, sum_int
    
  return X

nr1 = 4
nr2 = 123
           
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' ']
n_chars = len(alphabet)
print(n_chars)

X,Y,sum_int = encode_numbers(nr1, nr2, True)
print("X:", X)
print("Y:", Y)
print(X.shape)

11
X: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
Y: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
(8, 11)


In [29]:
max_lenght = 3
n_numbers = 10000

X_input = np.zeros((n_numbers, 2)).astype(int)

for i in range(n_numbers):
  X_input[i,0] = np.random.randint(10**max_lenght)
  X_input[i,1] = np.random.randint(10**max_lenght)

print(X_input)
  

[[173 488]
 [650 459]
 [343 713]
 ...
 [922 890]
 [615 969]
 [860 524]]


In [0]:
          
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' ']
n_chars = len(alphabet)

XX_input = np.zeros((n_numbers, (max_lenght+1)*2, n_chars))
YY = np.zeros((n_numbers, max_lenght+1, n_chars))

for i in range(n_numbers):
  XX_input[i], YY[i],sum_result = encode_numbers(X_input[i,0], X_input[i,1], True)



### Step2: Configure and Fit a seq2seq LSTM Model


In [59]:
# define LSTM configuration
# n_batch = 10
# n_epoch = 30
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' ']
n_chars = len(alphabet)
largest = 255

n_numbers =2
n_in_seq_length = n_numbers * ceil(log10(largest+1)) + n_numbers
# n_in_seq_length = 3

n_out_seq_length = ceil(log10(n_numbers * (largest+1))) + 1 
# create LSTM

model = Sequential()
model.add(LSTM(100, input_shape=(n_in_seq_length, n_chars)))
# model.add(LSTM(100, input_shape=XX_input.shape))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 100)               44800     
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 4, 100)            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 4, 50)             30200     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 4, 11)             561       
Total params: 75,561
Trainable params: 75,561
Non-trainable params: 0
_________________________________________________________________
None


#### Train model:

In [60]:
# 	model.fit(XX_input, YY, epochs=1, batch_size=n_batch)
# 	model.fit(XX_input, YY, epochs=400)
# 	model.fit(XX_input, YY, epochs=100)

model.fit(XX_input, YY, epochs=200)


Epoch 1/200
3000/3000 [==============================] - 5s 2ms/step - loss: 2.2175 - acc: 0.1983
Epoch 2/200
3000/3000 [==============================] - 2s 543us/step - loss: 1.8992 - acc: 0.2552
Epoch 3/200
3000/3000 [==============================] - 2s 532us/step - loss: 1.8667 - acc: 0.2679
Epoch 4/200
3000/3000 [==============================] - 2s 520us/step - loss: 1.8379 - acc: 0.2858
Epoch 5/200
3000/3000 [==============================] - 2s 541us/step - loss: 1.8223 - acc: 0.2886
Epoch 6/200
3000/3000 [==============================] - 2s 541us/step - loss: 1.8001 - acc: 0.3037
Epoch 7/200
3000/3000 [==============================] - 2s 539us/step - loss: 1.7633 - acc: 0.3224
Epoch 8/200
3000/3000 [==============================] - 2s 541us/step - loss: 1.7084 - acc: 0.3507
Epoch 9/200
3000/3000 [==============================] - 2s 533us/step - loss: 1.6635 - acc: 0.3641
Epoch 10/200
3000/3000 [==============================] - 2s 541us/step - loss: 1.6131 - acc: 0.3857
E

### Step3: Evaluate model

In [61]:
# def invert(value, n_numbers, largest):
#   print(value)
#   return round(value * float(largest * n_numbers))
def decode(value, n_numbers):
  print(value)
  return 1

def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

def integer_combine(n_array):
  result = 0
  for num in n_array:
    if(num!=10):
      result = result * 10 + round(num)
  return result

# def decode2(value, n_numbers):
#   print(value)
#   return 1

y = YY.astype(int)
# print(y)

n_numbers = 1000
print(y.shape)
result1 = [integer_combine(one_hot_decode(x)) for x in y]
print(result1)
# [decode(x, n_numbers) for x in y]


# print([invert(x, n_numbers, largest) for x in y])
# print(invert(x, n_numbers, largest) for x in y])

(3000, 4, 11)
[661, 1109, 1056, 134, 1556, 553, 93, 538, 831, 869, 878, 1302, 1262, 921, 659, 229, 887, 1445, 684, 318, 1240, 142, 652, 924, 376, 650, 1363, 1415, 592, 1461, 1053, 1663, 642, 536, 866, 1568, 887, 1399, 567, 935, 874, 303, 1302, 1485, 912, 1497, 1738, 239, 1153, 1544, 838, 1060, 234, 74, 517, 606, 312, 1107, 1351, 1421, 837, 761, 1507, 960, 906, 633, 1356, 1118, 838, 1361, 1100, 852, 1037, 1883, 1234, 1190, 306, 1224, 137, 415, 1354, 1078, 935, 1491, 535, 1521, 1432, 322, 1068, 452, 1486, 1336, 994, 1830, 464, 1228, 1761, 421, 1146, 1291, 711, 1053, 284, 759, 940, 410, 1192, 871, 1013, 996, 1675, 104, 288, 1885, 1706, 845, 90, 1109, 1364, 962, 866, 843, 1157, 1333, 1265, 1302, 939, 313, 985, 1031, 1468, 391, 921, 892, 1141, 463, 914, 1129, 1869, 1386, 879, 905, 979, 491, 980, 1144, 697, 1628, 244, 1366, 558, 1033, 1294, 1158, 976, 1611, 1683, 1515, 1603, 876, 895, 1787, 1545, 1054, 1411, 941, 1499, 1147, 1561, 525, 1154, 1208, 1790, 1241, 251, 1232, 977, 910, 1518, 995, 

In [63]:
from random import seed
from random import randint
from numpy import array
from math import ceil
from math import log10
from math import sqrt
from numpy import argmax
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector


n_numbers = 3000

# evaluate on some new patterns
          
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' ']
n_chars = len(alphabet)

XX_input = np.zeros((n_numbers, (max_lenght+1)*2, 11))
YY = np.zeros((n_numbers, max_lenght+1, 11))
print(YY)

for i in range(n_numbers):
  XX_input[i], YY[i], sum_result = encode_numbers(X_input[i,0], X_input[i,1], True)

  
# X, y = generate_data(n_samples, n_numbers, largest, alphabet)
# result = model.predict(X, batch_size=n_batch, verbose=0)
result = model.predict(XX_input)

# print(XX_input[0])

# print(result[0])
# print(integer_combine(one_hot_decode(result[0])))
# print(YY[0])
# print(integer_combine(one_hot_decode(YY[0])))


# calculate error
# expected = YY
# predicted = [invert(x, alphabet) for x in result]
predicted = [integer_combine(one_hot_decode(x)) for x in result]
expected = [integer_combine(one_hot_decode(x)) for x in YY]



rmse = sqrt(mean_squared_error(expected, predicted))
print('RMSE: %f' % rmse)

# # show some examples
for i in range(20):
	print('Expected=%s, Predicted=%s' % (expected[i], predicted[i]))
	print('Difference=%s' % (abs(expected[i] - predicted[i])))
  

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
RMSE: 2.569176
Expected=661, Predicted=661
Difference=0
Expected=1109, Predicted=1109
Difference=0
Expected=1056, Predicted=1056
Difference=0
Expected=134, Predicted=134
Difference=0
Expected=1556, Predicted=1556
Difference=0
Expected=553, Predicted=553
Difference=0
Expected=93, Predicted=93
Difference=0
Expected=538, Predicted=538
Difference=0
Expected=83

In [0]:
# TODO: test????
# loss, acc = model.evaluate(test_images, test_labels)

### Save (and load) model


In [0]:
from keras.models import load_model

model.save('RNN_sum_model.h5')  # creates a HDF5 file 'my_model.h5'

# Forloading model:
# model = load_model('RNN_sum_model.h5')
# model.summary()